## 0 Init

In [1]:
# Cancel the comment to install all the packages and libraries needed.
# ! pip install rasterio matplotlib rasterstats ipynbname imageio tqdm rasterstats
# ! pip install numpy==1.24.4CURR_PATH
# ! pip install libpysal
# ! pip install geopandas libpysal esda matplotlib
# ! pip install seaborn
# ! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

# Configuration
from pathlib import Path
import sys

CURR_PATH = Path().resolve()            # current file path
REPO_PATH = CURR_PATH.parent            # current repository path
DATA_PATH = REPO_PATH / "data"          # path for saving the data
DEMO_PATH = DATA_PATH / "demo-data"     # path for demo purpose 

SRC_PATH = REPO_PATH / "src"    # path for other sources
sys.path.append(str(SRC_PATH))  # add src to system path to import custom functions


## 1 Data Preparation: Temporal & Spatial Aggregation

- Initial data resolution:
    - Time: daily
    - Space: cell (mesh grid)

- Total four types of data with different aggregation level:
    1. Cell level & Daily Data (original parquet file)
    2. Cell level & Monthly Data
    3. Sub-region level & Daily Data
    4. Sub-region level & Monthly Data

### Addis Ababa

In [39]:
import numpy as np
import pandas as pd
# Read the Addis Ababa data
full_df = pd.read_parquet(DATA_PATH / "temp" / "full_addis_df.parquet", engine="pyarrow")
full_df['month'] = full_df['date'].dt.to_period('M')

# Original dataset: daily / cell resolution
daily_cell_ori = full_df

In [41]:
# type(full_df)
# full_df

First, generate the cell level, monthly data and save as csv file.

In [17]:
# features that need mean when aggregate from day to month level
time_agg_mean_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

monthly_cell = daily_cell_ori.groupby(['geom_id', 'month', 'ADM3_EN'])[time_agg_mean_feature].mean().reset_index()
monthly_cell.to_csv(DATA_PATH / "temp" / 'addis_monthly_cell.csv', index=False)

In [23]:
# monthly_cell.head()

Second, generate the sub-district level, daily data and save as csv file.

In [21]:
spatial_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]
space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

daily_adm3_sum = daily_cell_ori.groupby(['date', 'ADM3_EN'])[spatial_agg_sum_feature].sum().reset_index()
daily_adm3_avg = daily_cell_ori.groupby(['date', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()
daily_adm3 = daily_adm3_avg.merge(daily_adm3_sum, on=['date', 'ADM3_EN'], how='left')
daily_adm3.to_csv(DATA_PATH / "temp" / 'addis_daily_adm3.csv', index=False)

In [25]:
# daily_adm3.head()

Third, generate the sub-district level, monthly data and save as csv file.

In [26]:
space_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

monthly_adm3_sum = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_sum_feature].sum().reset_index()
monthly_adm3_avg = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()

In [28]:
monthly_adm3 = monthly_adm3_avg.merge(monthly_adm3_sum, on=['month', 'ADM3_EN'], how='left')
monthly_adm3.to_csv(DATA_PATH / "temp" / 'addis_monthly_adm3.csv', index=False)

In [31]:
# monthly_adm3

### Baghdad

In [45]:
import numpy as np
import pandas as pd
# Read the data
full_df = pd.read_parquet(DATA_PATH / "temp" / "full_baghdad_df.parquet", engine="pyarrow")
full_df['month'] = full_df['date'].dt.to_period('M')
full_df = full_df.rename(columns={'temp_mean': 'LST_day_mean'})   # unify to LST_day_mean

# Original dataset: daily / cell resolution
daily_cell_ori = full_df

In [43]:
full_df

,geom_id,no2_mean,pop_sum_m,NTL_mean,road_len,road_share,poi_count,poi_share,lu_industrial_area,lu_industrial_share,...,no2_lag1,no2_neighbor_lag1,cloud_category,LST_day_mean,landcover_2023,Shape_Leng,Shape_Area,ADM3_EN,ADM3_PCODE,month
0,0,0.000111,44.653709,2.088287,0.000000,0.000000,0,0.0,0.0,0.0,...,NaN,NaN,0.0,11.831877,12.0,1.384347,0.047983,Al-Latifya,IQG08Q05N03,2023-01
1,0,0.000225,44.653709,2.127017,0.000000,0.000000,0,0.0,0.0,0.0,...,0.000111,0.000109,2.0,16.110000,12.0,1.384347,0.047983,Al-Latifya,IQG08Q05N03,2023-01
2,0,0.000188,44.653709,NaN,0.000000,0.000000,0,0.0,0.0,0.0,...,0.000225,0.000246,0.0,12.601188,12.0,1.384347,0.047983,Al-Latifya,IQG08Q05N03,2023-01
3,0,0.000169,44.653709,NaN,0.000000,0.000000,0,0.0,0.0,0.0,...,0.000188,0.000189,1.0,NaN,12.0,1.384347,0.047983,Al-Latifya,IQG08Q05N03,2023-01
4,0,0.000151,44.653709,14.344322,0.000000,0.000000,0,0.0,0.0,0.0,...,0.000169,0.000173,1.0,NaN,12.0,1.384347,0.047983,Al-Latifya,IQG08Q05N03,2023-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4478101,6125,0.000166,289.352356,3.889121,3077.058813,0.000117,0,0.0,0.0,0.0,...,0.000365,0.000355,1.0,7.110000,12.0,1.091430,0.040133,Al-Tarmiya,IQG08Q02N04,2024-12
4478102,6125,0.000076,289.352356,39.110039,3077.058813,0.000117,0,0.0,0.0,0.0,...,0.000166,0.000179,2.0,14.770000,12.0,1.091430,0.040133,Al-Tarmiya,IQG08Q02N04,2024-12
4478103,6125,0.000216,289.352356,30.253170,3077.058813,0.000117,0,0.0,0.0,0.0,...,0.000076,0.000076,1.0,13.430000,12.0,1.091430,0.040133,Al-Tarmiya,IQG08Q02N04,2024-12
4478104,6125,0.000135,289.352356,37.280602,3077.058813,0.000117,0,0.0,0.0,0.0,...,0.000216,0.000247,0.0,16.370000,12.0,1.091430,0.040133,Al-Tarmiya,IQG08Q02N04,2024-12


First, generate the cell level, monthly data and save as csv file.

In [46]:
# features that need mean when aggregate from day to month level
time_agg_mean_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

monthly_cell = daily_cell_ori.groupby(['geom_id', 'month', 'ADM3_EN'])[time_agg_mean_feature].mean().reset_index()
monthly_cell.to_csv(DATA_PATH / "temp" / 'baghdad_monthly_cell.csv', index=False)

In [ ]:
# monthly_cell.head()

Second, generate the sub-district level, daily data and save as csv file.

In [47]:
spatial_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]
space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

daily_adm3_sum = daily_cell_ori.groupby(['date', 'ADM3_EN'])[spatial_agg_sum_feature].sum().reset_index()
daily_adm3_avg = daily_cell_ori.groupby(['date', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()
daily_adm3 = daily_adm3_avg.merge(daily_adm3_sum, on=['date', 'ADM3_EN'], how='left')
daily_adm3.to_csv(DATA_PATH / "temp" / 'baghdad_daily_adm3.csv', index=False)

In [49]:
daily_adm3

,date,ADM3_EN,LST_day_mean,no2_mean,no2_lag1,no2_neighbor_lag1,NTL_mean,TCI,pop_sum_m,road_len,...,lu_residential_area,lu_retail_area,lu_farmland_area,lu_farmyard_area,road_primary_len,road_motorway_len,road_trunk_len,road_secondary_len,road_tertiary_len,road_residential_len
0,2023-01-01,Abu Ghraib,12.420119,0.095418,0.000000,0.000000,16517.924950,2.796893e+06,5.111909e+05,2.712699e+06,...,2.436915e+07,0.0,3.297929e+07,8734.319149,72805.034518,134296.278767,0.000000,42553.429081,267884.154210,1.157555e+06
1,2023-01-01,Al-Fahama,12.771666,0.037629,0.000000,0.000000,7831.303788,3.661187e+07,1.019493e+06,1.771264e+06,...,3.925814e+07,0.0,1.072887e+06,0.000000,9374.771374,23504.337300,31207.293767,4882.810604,165507.886467,1.322054e+06
2,2023-01-01,Al-Jisr,13.350142,0.066570,0.000000,0.000000,6524.999007,5.499618e+06,2.537896e+05,6.784165e+05,...,9.367158e+06,0.0,1.893582e+07,0.000000,29253.292783,0.000000,31548.343818,14049.948014,52184.422521,3.835369e+05
3,2023-01-01,Al-Karrada Al-Sharqia,13.822724,0.043889,0.000000,0.000000,10284.304605,2.778583e+07,6.804654e+05,1.657440e+06,...,2.692580e+07,0.0,6.301526e+06,0.000000,86620.551798,32598.567966,3730.427599,40736.417691,118433.362443,8.482135e+05
4,2023-01-01,Al-Latifya,12.399308,0.072134,0.000000,0.000000,5038.490275,3.488123e+06,1.598628e+05,8.827485e+05,...,2.360183e+06,0.0,2.155193e+08,0.000000,7899.550833,58368.588112,39685.810012,40019.461397,59683.649226,2.846482e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15346,2024-12-31,Markaz Al-Karkh,13.878697,0.000000,0.021264,0.021154,8148.203962,2.332277e+05,1.772346e+05,7.375371e+05,...,1.005766e+07,0.0,0.000000e+00,0.000000,71372.128640,14098.977033,0.000000,50811.351119,31200.153262,2.529159e+05
15347,2024-12-31,Markaz Al-Mada'in,13.722056,0.000000,0.406985,0.404960,7029.665967,1.069760e+05,1.636998e+05,7.846149e+05,...,1.549490e+07,0.0,1.479552e+08,0.000000,22978.465260,0.000000,42400.937644,0.000000,42946.439662,4.669922e+05
15348,2024-12-31,Markaz Al-Mahmoudiya,13.583168,0.000000,0.034868,0.034775,1850.581081,1.531018e+05,1.072385e+05,4.749349e+05,...,8.522331e+03,0.0,3.965280e+07,0.000000,9762.885545,7638.868219,9296.516086,2541.127138,57374.549910,2.398848e+05
15349,2024-12-31,Markaz Al-Thawra,13.638146,0.000000,0.046052,0.046124,10100.558853,3.099584e+05,1.156935e+06,1.571223e+06,...,3.415564e+07,0.0,0.000000e+00,0.000000,109349.280838,12861.668398,0.000000,49075.556344,167318.888287,1.085157e+06


Third, generate the sub-district level, monthly data and save as csv file.

In [50]:
space_agg_sum_feature = [
        'no2_mean', 'no2_lag1', 'no2_neighbor_lag1',
        'NTL_mean',
        'TCI',
        'pop_sum_m',  
        'road_len', 
        'poi_count', 'lu_industrial_area',
        'lu_commercial_area',  'lu_residential_area', 'lu_retail_area', 'lu_farmland_area', 
        'lu_farmyard_area', 
        'road_primary_len',
        'road_motorway_len', 'road_trunk_len',  'road_secondary_len', 'road_tertiary_len', 'road_residential_len',
         
]

space_agg_mean_feature = [
        # 'cloud_category', 
        'LST_day_mean', 
        # 'landcover_2023',       
]

monthly_adm3_sum = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_sum_feature].sum().reset_index()
monthly_adm3_avg = monthly_cell.groupby(['month', 'ADM3_EN'])[space_agg_mean_feature].mean().reset_index()

In [51]:
monthly_adm3 = monthly_adm3_avg.merge(monthly_adm3_sum, on=['month', 'ADM3_EN'], how='left')
monthly_adm3.to_csv(DATA_PATH / "temp" / 'baghdad_monthly_adm3.csv', index=False)

In [53]:
# monthly_adm3

## 2 Economic Models

### 2.1 Addis Ababa

### 2.2 Baghdad

#### 0) Data Loading

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

baghdad_df = pd.read_csv(DATA_PATH / "temp" / 'baghdad_monthly_adm3.csv')

y = baghdad_df['TCI']
X = baghdad_df.drop(columns=['month', 'ADM3_EN', 'TCI'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
baghdad_df.columns

Index(['month', 'ADM3_EN', 'LST_day_mean', 'LST_day_mean.1', 'no2_mean',
       'no2_lag1', 'no2_neighbor_lag1', 'NTL_mean', 'TCI', 'pop_sum_m',
       'road_len', 'poi_count', 'lu_industrial_area', 'lu_commercial_area',
       'lu_residential_area', 'lu_retail_area', 'lu_farmland_area',
       'lu_farmyard_area', 'road_primary_len', 'road_motorway_len',
       'road_trunk_len', 'road_secondary_len', 'road_tertiary_len',
       'road_residential_len', 'grassland_a', 'cropland_a', 'built_up_a',
       'snow_a', 'water_bod_a', 'wetland_a', 'sparse_veg_a', 'mangroves_a',
       'moss_a', 'unclassified_a', 'non_built_area'],
      dtype='object')

In [4]:
baghdad_df

,month,ADM3_EN,LST_day_mean,LST_day_mean.1,no2_mean,no2_lag1,no2_neighbor_lag1,NTL_mean,TCI,pop_sum_m,...,cropland_a,built_up_a,snow_a,water_bod_a,wetland_a,sparse_veg_a,mangroves_a,moss_a,unclassified_a,non_built_area
0,2023-01,Abu Ghraib,13.891630,31.603639,0.128318,0.130674,0.130740,10996.997104,3.090105e+06,5.111909e+05,...,3.369273e+08,1.154348e+08,0.0,3.189071e+07,211645.161290,2.326196e+08,0.0,0.0,0.0,4.409802e+08
1,2023-01,Al-Fahama,13.170177,50.629270,0.048078,0.048970,0.049028,9153.362860,2.309475e+07,1.019493e+06,...,5.763568e+07,2.228241e+07,0.0,5.555419e+06,20874.193548,3.988754e+07,0.0,0.0,0.0,7.745228e+07
2,2023-01,Al-Jisr,13.828319,39.756855,0.055382,0.056684,0.056633,5231.669979,5.913885e+06,2.537896e+05,...,6.779421e+07,2.256127e+07,0.0,6.747561e+06,29806.451613,4.985854e+07,0.0,0.0,0.0,9.247373e+07
3,2023-01,Al-Karrada Al-Sharqia,13.752808,25.661060,0.040241,0.041041,0.041008,7950.414601,5.217746e+07,6.804654e+05,...,3.788936e+07,1.582432e+07,0.0,3.981213e+06,53596.774194,3.041868e+07,0.0,0.0,0.0,5.644363e+07
4,2023-01,Al-Latifya,14.179958,22.240558,0.112341,0.114546,0.114575,4114.145839,2.600197e+06,1.598628e+05,...,2.667754e+08,9.610082e+07,0.0,2.570384e+07,196951.612903,1.956018e+08,0.0,0.0,0.0,3.635005e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,2024-12,Markaz Al-Karkh,13.587203,29.759993,0.017129,0.016944,0.016914,5899.708834,2.890384e+07,1.772346e+05,...,1.573453e+07,5.987877e+06,0.0,1.757529e+06,27474.193548,1.144857e+07,0.0,0.0,0.0,2.195085e+07
500,2024-12,Markaz Al-Mada'in,14.087136,42.187065,0.116569,0.115825,0.115772,4075.326984,3.457018e+06,1.636998e+05,...,1.300586e+08,4.663562e+07,0.0,1.178572e+07,46958.064516,9.405811e+07,0.0,0.0,0.0,1.762193e+08
501,2024-12,Markaz Al-Mahmoudiya,13.928305,33.394826,0.020850,0.020855,0.020854,1436.757802,2.129347e+06,1.072385e+05,...,3.867399e+07,1.358959e+07,0.0,4.030290e+06,13322.580645,2.612048e+07,0.0,0.0,0.0,4.995571e+07
502,2024-12,Markaz Al-Thawra,13.546974,50.605822,0.029951,0.029508,0.029561,6268.450398,2.285307e+07,1.156935e+06,...,2.951323e+07,1.495888e+07,0.0,2.625861e+06,19000.000000,2.021855e+07,0.0,0.0,0.0,3.827668e+07


#### 1) Simple Linear Model

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_x.fit_transform(X_train)
X_test_scaled = scaler_x.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).ravel()

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train_scaled)

y_pred_scaled = lr_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test_scaled, y_pred_scaled)
r2 = r2_score(y_test_scaled, y_pred_scaled)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'R-squared (R²): {r2:.4f}')


Mean Squared Error (MSE): 0.0074
R-squared (R²): 0.8020


#### 2) Log-Log Linear Regression

In [5]:
import numpy as np

X_train_log = np.log1p(X_train)
X_test_log = np.log1p(X_test)

y_train_log = np.log1p(y_train.values.reshape(-1, 1)).ravel()
y_test_log = np.log1p(y_test.values.reshape(-1, 1)).ravel()


In [6]:
y_train_log

array([12.66678772, 16.67164869, 16.2494287 ,  0.        ,  7.78451692,
       16.36426491, 15.5010652 , 16.12997938, 17.12984042, 15.13524303,
       15.20792316, 13.88190865, 16.73760834, 14.4829227 , 18.0791987 ,
       13.25283413, 12.63589846, 16.70367188, 13.81310637, 17.32262931,
       16.86345965, 14.54732133, 12.88332097, 15.02614878, 17.98828012,
        8.84209905, 16.67671657, 15.4057291 ,  0.        , 14.71776424,
       13.30939266,  7.57183308, 14.94018334, 16.46834702, 15.56926537,
       16.4792228 , 16.4425988 , 16.67147068, 17.8291768 , 14.69449763,
       17.90211867, 17.53672784, 16.94709136, 13.73913283,  6.37514163,
       12.55948226, 14.44434699, 15.57510758, 16.50537524, 14.33839997,
       16.55781746, 16.74565278, 13.50381171, 17.87633995, 16.90157604,
       10.55243336, 11.67175014, 17.77016118, 15.4300757 , 14.24400703,
       13.56688175, 16.43850535, 16.46056415,  0.        , 15.42297477,
        0.        , 17.11275337, 16.71499927, 14.5132728 , 16.76

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_x.fit_transform(X_train_log)
X_test_scaled = scaler_x.transform(X_test_log)

y_train_scaled = scaler_y.fit_transform(y_train_log.reshape(-1, 1)).ravel()
y_test_scaled = scaler_y.transform(y_test_log.reshape(-1, 1)).ravel()


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train_scaled)

y_pred_scaled = lr_model.predict(X_test_scaled)


In [9]:

y_pred_log = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()

y_pred = np.expm1(y_pred_log)


In [10]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'R-squared (R²): {r2:.4f}')


Mean Squared Error (MSE): 546972019768371.7500
R-squared (R²): -0.1465


In [11]:
y_pred

array([8.13826268e+06, 3.12242201e+06, 4.17491175e+05, 6.98563605e+05,
       1.45795907e+05, 5.81015819e+06, 1.28618328e+06, 4.75220559e+07,
       3.59129757e+04, 7.02509238e+04, 1.21235710e+06, 4.49251863e+04,
       2.15901914e+05, 1.19733198e+06, 1.93450339e+05, 7.07349058e+04,
       1.83544861e+07, 3.98626933e+07, 1.02995936e+07, 7.33109422e+06,
       5.20045586e+05, 4.43400407e+05, 1.06649718e+05, 2.36497857e+05,
       1.46819349e+06, 2.35119760e+07, 4.28975843e+06, 2.49605420e+07,
       4.20681385e+05, 8.56843001e+03, 6.29729994e+05, 1.65073021e+05,
       2.12415816e+06, 8.42763117e+07, 4.03126531e+06, 5.00070687e+07,
       5.84644327e+04, 8.08905714e+05, 8.82878895e+06, 2.06590106e+03,
       7.52693262e+03, 1.48322166e+03, 1.83005218e+06, 2.79051190e+06,
       2.20833140e+07, 9.99276154e+05, 2.65769535e+04, 1.18781485e+05,
       1.98502683e+06, 3.00451577e+04, 2.61424025e+04, 5.36476436e+05,
       2.88150813e+06, 1.74292957e+06, 3.73855472e+05, 1.28928285e+04,
      